In [1]:
import rds_config
from sqlalchemy import create_engine

#rds settings
db_host  = rds_config.db_host
db_username = rds_config.db_username
db_password = rds_config.db_password
db_name = rds_config.db_name

#한글 깨짐 현상 해결
#http://yongho1037.tistory.com/570
SQLALCHEMY_DATABASE_URI = 'mysql+pymysql://' + db_username + ':' + db_password + '@' + db_host + '/' + db_name + '?charset=utf8'

engine = create_engine(SQLALCHEMY_DATABASE_URI)
connection = engine.connect()

In [2]:
import pandas as pd
sql = "SELECT date, substring(date, 6,2) as month, media, title, org_count, naver_count, org_count+naver_count as total_count, section FROM newscrawl WHERE org_count>0 or naver_count>0 "
df = pd.read_sql(sql, connection)
#pd.read_csv("dcdata10.csv")

In [3]:
len(df)

445504

In [4]:
df['title']=df['title'].str.replace(r'\[[A-Za-z가-힣0-9 ]+\]|\([A-Za-z가-힣0-9 ]+\)|\<[A-Za-z가-힣0-9 ]+\>','')

In [5]:
%%time
from konlpy.tag import Mecab
mecab = Mecab()

def mecab_words(sentence):
    word_list = mecab.nouns(sentence)
    #return " ".join(word_list)
    return " ".join(filter(lambda x: len(x)>1, word_list))
    #return filter(lambda x: len(x)>1, " ".join(word_list))

df['title_me_noun'] = df['title'].apply(mecab_words)

CPU times: user 31.7 s, sys: 156 ms, total: 31.9 s
Wall time: 33 s


In [6]:
df_by_month = df.groupby(['month'])
df_by_media_month = df.groupby(['month', 'media'])

In [7]:
from collections import Counter

by_me_month = df_by_month['title_me_noun'].apply(list)
result_by_me_month = pd.Series()

for month, words in by_me_month.iteritems():
    result_by_me_month.set_value(month, Counter(" ".join(words).split(" ")))

In [18]:
for month, words in result_by_me_month.iteritems():
    print(month, words.most_common(50))

01 [('한국', 1129), ('대통령', 906), ('중국', 828), ('서울', 692), ('위안부', 671), ('경제', 671), ('더민주', 652), ('정부', 623), ('시장', 598), ('북한', 589), ('세계', 587), ('논란', 572), ('안철수', 522), ('미국', 511), ('한파', 466), ('의원', 455), ('일본', 443), ('가능', 442), ('정치', 426), ('기업', 415), ('청년', 406), ('공항', 406), ('공개', 399), ('사망', 397), ('합의', 388), ('핵실험', 386), ('올해', 386), ('영입', 383), ('우리', 383), ('제주', 371), ('대북', 371), ('경찰', 370), ('대표', 362), ('탈당', 352), ('여성', 351), ('제재', 346), ('새해', 343), ('총선', 341), ('국회', 333), ('누리', 333), ('김종인', 324), ('이유', 316), ('김무성', 313), ('조선', 311), ('최고', 311), ('지원', 309), ('문재인', 306), ('개발', 305), ('사람', 303), ('교수', 302)]
02 [('한국', 910), ('개성공단', 769), ('대통령', 737), ('중국', 681), ('사드', 668), ('정부', 666), ('북한', 659), ('미사일', 659), ('서울', 560), ('기업', 543), ('시장', 531), ('더민주', 531), ('세계', 510), ('가능', 496), ('필리버스터', 477), ('제재', 473), ('논란', 455), ('경제', 455), ('발사', 452), ('의원', 447), ('배치', 427), ('공천', 421), ('대북', 420), ('총선', 414), ('공개', 394), 

In [43]:
df.reset_index().to_json('data/mecab.json',orient='records')

In [11]:
%%time
from collections import Counter

## df 방식과 counter 방식의 현저한 속도차
#result = df['title_noun'].head(20000).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)
#result.sort_values(axis=0, ascending=False)
r1 = df['title_me_noun'].tolist()
result = Counter(" ".join(r1).split(" "))

CPU times: user 744 ms, sys: 96 ms, total: 840 ms
Wall time: 836 ms


In [12]:
result.most_common(100)

[('대통령', 16368),
 ('한국', 10848),
 ('정부', 6978),
 ('서울', 6886),
 ('중국', 6595),
 ('논란', 6259),
 ('최순실', 6254),
 ('기업', 5591),
 ('세계', 5500),
 ('트럼프', 5412),
 ('시장', 5311),
 ('경찰', 5267),
 ('검찰', 5104),
 ('여성', 5066),
 ('사드', 4986),
 ('경제', 4839),
 ('가능', 4712),
 ('의혹', 4690),
 ('수사', 4630),
 ('사망', 4569),
 ('더민주', 4492),
 ('대표', 4463),
 ('의원', 4447),
 ('조사', 4444),
 ('박근혜', 4257),
 ('공개', 4241),
 ('시간', 4196),
 ('북한', 4171),
 ('국회', 4062),
 ('미국', 4014),
 ('일본', 3912),
 ('국민', 3912),
 ('우리', 3766),
 ('지원', 3675),
 ('조선', 3675),
 ('정치', 3662),
 ('이유', 3483),
 ('최고', 3429),
 ('최대', 3394),
 ('부산', 3344),
 ('사람', 3305),
 ('새누리', 3268),
 ('추진', 3136),
 ('총선', 3086),
 ('국민의당', 3052),
 ('후보', 3027),
 ('사고', 2991),
 ('시민', 2987),
 ('전국', 2981),
 ('개발', 2941),
 ('구속', 2882),
 ('지진', 2865),
 ('청와대', 2863),
 ('투자', 2860),
 ('사업', 2788),
 ('청년', 2755),
 ('삼성', 2747),
 ('세월호', 2736),
 ('국내', 2735),
 ('탄핵', 2715),
 ('대선', 2696),
 ('문제', 2690),
 ('시대', 2677),
 ('테러', 2656),
 ('학교', 2651),
 ('규모', 2650)

In [15]:
%%time
from konlpy.tag import Twitter
twitter = Twitter()
def words(sentence):
    word_list = twitter.nouns(sentence)
    return " ".join(word_list)

df['title_noun'] = df['title'].apply(words)
#df.reset_index().to_json('data/twitter.json',orient='records')

CPU times: user 3min 58s, sys: 592 ms, total: 3min 59s
Wall time: 3min 54s


In [15]:
df[df['month']=='01']['title']

0            정의장 "5일까지 선거구 획정안 요청"…일부 시군구 분할허용
1                與 "정의장, 선거구 획정안만 직권상정하는 건 불가"
2               野, 정의장 획정기준에 "게리맨더링 허용한 나쁜 선례"
3                  경기도·도교육청 사상 첫 준예산 사태…혼란 불가피
4                 준예산사태 경기도 보육대란 현실화…35만명 지원중단
5               오바마, 인턴과의 대화서 "슬프지만 나도 죽음을 상상"
6                   정의장 제시 획정기준에 따른 선거구조정 시나리오
7             2016년 '제야의 종소리'와 함께 활짝…"새해엔 행복만"
8                  '해맞이 가던 차량서 불'…영동고속도로 정체 지속
9                  '붉은 원숭이의 해' 대한민국 첫둥이 우렁찬 울음
10                 경기도의회 여야 거친 몸싸움 속 예산안 처리 불발
11              이재정 교육감 "준예산 사태로 '식물교육' 상태 초래"
12         미국 총기 면허 1위 텍사스, 2016년부터 '총잡이 전성시대'
13          北김정은에 각국 지도자 연하장…북한, 中보다 러시아 먼저 호명
14               교황 "가끔 화날 때도 있지만, 누구를 해치진 않아"
15              마이크로RNA 생성 관여하는 드로셔 3차원구조 첫 규명
16                    하와이, 새해부터 21세 미만 흡연 `불법'
17            미 육군 현역장교 "미군 주둔은 미국의 이익" 트럼프 비판
18                 '붉은 원숭이의 해' 대한민국 첫둥이 우렁찬 울음
19                   "46년간 페이스북 친구?" 한때 서비스 오류
20        美 '아이오와 결전' 한달 앞으로…대선 1차관문서 '이변' 있을까
21           